# Obtención y Limpieza de los datos

Proyecto 1 - Data Science

Edwin Ortega 22305 - Esteban Zambrano 22119 - Diego García 22404

### Configuración e importaciones

In [29]:
from pathlib import Path
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
from xlrd import XLRDError

# Paths
DATA_RAW = Path("../data/raw_data")
DATA_INTERIM = Path("../data/provisional")
DATA_PROCESSED = Path("../data/procesada")
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Archivos esperados
DEPARTAMENTOS = [
    "AltaVerapaz","BajaVerapaz","Chimaltenango","Chiquimula","CiudadCapital",
    "ElProgreso","Escuintla","Guatemala","Huehuetenango","Izabal","Jalapa",
    "Jutiapa","Peten","Quetzaltenango","Quiche","Retalhuleu","Sacatepequez",
    "SanMarcos","SantaRosa","Solola","Suchitepequez","Totonicapan","Zacapa"
]

### Consolidación cruda

In [ ]:
# Columnas esperadas (en MAYÚSCULAS y sin acentos)
EXPECTED = {
    "CODIGO","DISTRITO","DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION",
    "TELEFONO","SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA","STATUS",
    "MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"
}

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Aplana encabezados raros, quita 'Unnamed', pasa a string+UPPER+sin acentos.
       Si la primera fila parece ser header real, la usa como encabezados"""
    # Aplastar MultiIndex si hay
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join(map(str, t)).strip() for t in df.columns.values]

    # Normalizar encabezados actuales
    df.columns = [unidecode(str(c)).upper().strip() for c in df.columns]
    # Quitar columnas basura
    df = df.loc[:, ~df.columns.str.startswith("UNNAMED")]

    if len(df) > 0:
        first = [unidecode(str(x)).upper().strip() for x in df.iloc[0].tolist()]
        if set(EXPECTED).issubset(set(first)):
            df = df.iloc[1:].copy()
            df.columns = first
    return df

def _pick_html_table(fp: str) -> pd.DataFrame:
    """Lee todas las tablas HTML y escoge la mejor según coincidencia con EXPECTED y tamaño"""
    tables = pd.read_html(fp)
    best = None
    best_score = (-1, -1)

    for t in tables:
        t = t.astype(str)
        t = _normalize_columns(t)
        cols = set(t.columns)
        matches = len(cols & EXPECTED)
        score = (matches, len(t))
        if score > best_score:
            best, best_score = t, score

    if best is None:
        raise ValueError(f"{fp} no contiene tablas HTML aprovechables.")
    return best

def read_one_excel(dep: str) -> pd.DataFrame:
    """Lee un archivo por departamento desde DATA_RAW"""
    for ext in (".xlsx", ".xls"):
        fp = DATA_RAW / f"{dep}{ext}"
        if not fp.exists():
            continue

        try:
            if ext == ".xlsx":
                df = pd.read_excel(fp, dtype=str, engine="openpyxl")
            else:  # ".xls"
                try:
                    df = pd.read_excel(fp, dtype=str, engine="xlrd")
                except (ValueError, XLRDError):
                    # .xls que en realidad es HTML
                    df = _pick_html_table(str(fp))
        except Exception as e:
            raise RuntimeError(f"Error leyendo {fp}: {e}")

        # Normalizar encabezados y limpiar ruidos típicos
        df = _normalize_columns(df)

        # Marcar strings vacíos como NA y eliminar filas totalmente vacías
        df = df.replace(r'^\s*$', pd.NA, regex=True)
        df = df.dropna(how="all").reset_index(drop=True)

        # Convertir a Strings para evitar 'nan' literales
        for c in df.columns:
            df[c] = df[c].astype("string")

        df["DEPARTAMENTO_ORIGEN"] = dep
        return df

    raise FileNotFoundError(f"No se encontró {dep}.xlsx ni {dep}.xls en {DATA_RAW}")

# LECTURA de todos los departamentos
dfs = [read_one_excel(dep) for dep in DEPARTAMENTOS]
raw = pd.concat(dfs, ignore_index=True, sort=False)

# Normalizar por si acaso tras el concat
raw.columns = [unidecode(str(c)).upper().strip() for c in raw.columns]

# Renombrado estándar a minúsculas finales
col_map = {
    "CODIGO":"codigo","DISTRITO":"distrito","DEPARTAMENTO":"departamento",
    "MUNICIPIO":"municipio","ESTABLECIMIENTO":"establecimiento","DIRECCION":"direccion",
    "TELEFONO":"telefono","SUPERVISOR":"supervisor","DIRECTOR":"director",
    "NIVEL":"nivel","SECTOR":"sector","AREA":"area","STATUS":"status",
    "MODALIDAD":"modalidad","JORNADA":"jornada","PLAN":"plan","DEPARTAMENTAL":"departamental",
    "DEPARTAMENTO_ORIGEN":"departamento_origen"
}
raw = raw.rename(columns=col_map)

# Normaliza vacíos y 'nan'/'None' en TODAS las columnas a NA reales
raw = raw.replace(r'^\s*$', pd.NA, regex=True)
raw = raw.replace(r'^\s*(nan|none|null)\s*$', pd.NA, regex=True)

# Quita filas totalmente vacías
raw = raw.dropna(how="all")

# Exige que haya al menos 3 campos NO nulos (ignorando 'departamento_origen')
core_cols = [c for c in raw.columns if c != "departamento_origen"]
raw = raw[ raw[core_cols].notna().sum(axis=1) >= 3 ]

# Si existen, exige además 'codigo' y 'establecimiento'
if {"codigo","establecimiento"}.issubset(raw.columns):
    raw = raw.dropna(subset=["codigo","establecimiento"], how="any")

# Guarda CSV's
raw.to_csv(DATA_INTERIM / "establecimientos_diversificado_raw_concat.csv",
           index=False, encoding="utf-8")

print("Shape crudo concatenado:", raw.shape)
print("Columnas:", sorted(raw.columns))
print("Conteo por departamento:")
print(raw["departamento_origen"].value_counts(dropna=False))


Shape crudo concatenado: (6599, 18)
Columnas: ['area', 'codigo', 'departamental', 'departamento', 'departamento_origen', 'direccion', 'director', 'distrito', 'establecimiento', 'jornada', 'modalidad', 'municipio', 'nivel', 'plan', 'sector', 'status', 'supervisor', 'telefono']
Conteo por departamento:
departamento_origen
Guatemala         1038
CiudadCapital      866
SanMarcos          432
Escuintla          393
Quetzaltenango     365
Chimaltenango      304
Jutiapa            296
Suchitepequez      296
Huehuetenango      295
AltaVerapaz        294
Izabal             273
Retalhuleu         272
Peten              270
Sacatepequez       208
Quiche             184
Chiquimula         136
SantaRosa          133
Jalapa             121
Solola             111
ElProgreso          97
BajaVerapaz         94
Zacapa              70
Totonicapan         51
Name: count, dtype: int64


### Radiografía del crudo

In [ ]:
df = raw.copy()

# Estadisticas globales
print("Shape:", df.shape)
print("\nNulos por columna:")
print(df.isna().sum().sort_values(ascending=False))

print("\nCardinalidades:")
print(df.nunique().sort_values(ascending=False))

# --- Formatos en CODIGO ---
print("\nEjemplos de CODIGO:")
print(df['codigo'].dropna().sample(10, random_state=42).tolist())

pat_codigos = df['codigo'].dropna().apply(lambda x: bool(re.fullmatch(r'\d{2}-\d{2}-\d{4}-\d{2}', str(x))))
print(f"Formato válido CODIGO (%): {pat_codigos.mean()*100:.2f}%")

# Formatos en TELEFONO
print("\nEjemplos de TELEFONO:")
print(df['telefono'].dropna().sample(10, random_state=42).tolist())

pat_tel = df['telefono'].dropna().apply(lambda x: bool(re.fullmatch(r'\d{8}(-\d{8})?', str(x))))
print(f"Formato válido TELEFONO (%): {pat_tel.mean()*100:.2f}%")

# Formatos en DISTRITO
print("\nEjemplos de DISTRITO:")
print(df['distrito'].dropna().sample(10, random_state=42).tolist())

# Muestras problematicas
def find_with_regex(col, pattern, n=5):
    """Extrae hasta n ejemplos que coincidan con el patrón."""
    return df[df[col].astype(str).str.contains(pattern, na=False)][col].unique()[:n]

print("\nEjemplos con acentos:")
print(find_with_regex('establecimiento', r'[ÁÉÍÓÚáéíóú]'))

print("\nEjemplos con dobles espacios:")
print(find_with_regex('establecimiento', r'\s{2,}'))

print("\nEjemplos con paréntesis:")
print(find_with_regex('establecimiento', r'[\(\)]'))

print("\nEjemplos con abreviaturas (punto):")
print(find_with_regex('establecimiento', r'\.'))

# Tablas resumen para informe
summary_nulls = df.isna().sum().reset_index()
summary_nulls.columns = ['columna', 'nulos']

summary_card = df.nunique().reset_index()
summary_card.columns = ['columna', 'cardinalidad']

summary_formats = pd.DataFrame({
    'columna': ['codigo', 'telefono'],
    'porcentaje_formato_valido': [
        pat_codigos.mean()*100,
        pat_tel.mean()*100
    ]
})

# Guardar para Code Book
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
summary_nulls.to_csv(DATA_INTERIM / "resumen_nulos.csv", index=False)
summary_card.to_csv(DATA_INTERIM / "resumen_cardinalidades.csv", index=False)
summary_formats.to_csv(DATA_INTERIM / "resumen_formatos.csv", index=False)

print("\nRadiografía guardada en DATA_INTERIM/")


Shape: (6599, 18)

Nulos por columna:
telefono               46
director               26
direccion               2
codigo                  0
distrito                0
departamento            0
establecimiento         0
municipio               0
supervisor              0
nivel                   0
sector                  0
area                    0
status                  0
modalidad               0
jornada                 0
plan                    0
departamental           0
departamento_origen     0
dtype: int64

Cardinalidades:
codigo                 6599
direccion              4433
telefono               4207
director               3863
establecimiento        3786
distrito                622
supervisor              599
municipio               343
departamental            26
departamento             23
departamento_origen      23
plan                     12
jornada                   6
sector                    4
area                      3
modalidad                 2
nivel           